In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops import rnn, rnn_cell
import numpy as np
from python_speech_features import mfcc
%matplotlib inline
plt.style.use('ggplot')

In [2]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 13, frames = 25):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            #print fn
            sound_clip,s = librosa.load(fn)
            #print fn
            #print fn.split('/')[5].split('-')[0]
            label = fn.split('/')[5].split('-')[0]
            if(label=='n'):
                label=0
            if(label=='sp'):
                label=1
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                    mfccs.append(mfcc)
                    for ho in range(0,frames):
                        labels.append(label)         
    features = np.asarray(mfccs).reshape(len(mfccs)*frames,bands)
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [7]:


parent_dir = '/home/samim/audiotrainingset/'

tr_sub_dirs = ['/home/samim/audiotrainingset/training/']
tr_features,tr_labels = extract_features(parent_dir,tr_sub_dirs)
tr_labels = one_hot_encode(tr_labels)

ts_sub_dirs = ['/home/samim/audiotrainingset/testing/']
ts_features,ts_labels = extract_features(parent_dir,ts_sub_dirs)
ts_labels = one_hot_encode(ts_labels)

#for a singledata/audio sample
single_dirs = ['/home/samim/audiotrainingset/mydata/']
single_features,single_labels = extract_features(parent_dir,single_dirs)
single_labels = one_hot_encode(single_labels)


In [8]:
print np.shape(tr_features)
print np.shape(ts_features)
print np.shape(tr_labels)
print np.shape(ts_labels)
print np.shape(single_features)
print np.shape(single_labels)


(40450, 13)
(2700, 13)
(40450, 2)
(2700, 2)
(10700, 13)
(10700, 2)


In [14]:
print tr_features
#print ts_features
print single_features


[[  28.15841643  135.38226361  -57.07620374 ...,    1.56619996
    -7.09202049  -31.50583012]
 [  24.09893125  137.90576327 -114.65619221 ...,    2.76746728
   -13.80398321  -40.81468196]
 [ -88.58701285   93.49443242 -217.68395476 ...,   -7.49118347
     4.11981853  -29.8041856 ]
 ..., 
 [-365.26763342  221.46193591  -75.21476963 ...,  -32.07783477
    -9.96381627  -12.28702448]
 [-301.75540683  183.0349974   -67.3393457  ...,  -32.55723463
   -17.17785367   -1.30824554]
 [-277.44060537  159.09760949  -56.63679668 ...,  -33.11707145
   -18.06695419    3.47183371]]
[[ -9.20396521e+01   6.54939953e+01  -1.51110585e+01 ...,   9.08535566e-01
   -5.41304179e+00  -2.15575465e-01]
 [ -1.60120316e+02   6.54755996e+01  -1.51185839e+01 ...,   8.78619958e-01
   -5.38534111e+00  -2.18883964e-01]
 [ -8.48894135e+02  -4.76557669e-15   9.16569010e-14 ...,   1.05247149e-14
   -2.33339719e-14   1.04072339e-13]
 ..., 
 [ -3.56537802e+02   1.85095644e+02   1.66250130e+01 ...,  -2.10503693e+01
   -3.3575

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [20]:
mfcc_size=13
num_labels=2
batch_size = 1128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

	# Input data. For the training data, we use a placeholder that will be fed
	# at run time with a training minibatch.
	tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, mfcc_size))
	tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
	#tf_valid_dataset = tf.constant(valid_dataset)
	tf_test_dataset = tf.constant(ts_features)
	tf_test_dataset2 = tf.constant(single_features)
	# Variables.
	W1 = tf.Variable(tf.truncated_normal([mfcc_size, hidden_size]))
	b1 = tf.Variable(tf.zeros([hidden_size]))

	W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
	b2 = tf.Variable(tf.zeros([num_labels]))	
	#W1=tf.cast(W1, tf.float64)
	#b1=tf.cast(b1, tf.float64)
	#W2=tf.cast(W2, tf.float64)
	#b2=tf.cast(b2, tf.float64)
	# Training computation.
	y1 = tf.nn.relu( tf.matmul(tf_train_dataset, W1) + b1 )
	logits = tf.matmul(y1, W2) + b2
	
	loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
	
	# Optimizer.
	optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
	
	# Predictions for the training, validation, and test data.
	train_prediction = tf.nn.softmax(logits)

	#y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
	#valid_logits = tf.matmul(y1_valid, W2) + b2
	#valid_prediction = tf.nn.softmax(valid_logits)
	w1=tf.cast(W1, tf.float64)
	B1=tf.cast(b1, tf.float64)
	w2=tf.cast(W2, tf.float64)
	B2=tf.cast(b2, tf.float64)
	y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + B1)
	test_logits = tf.matmul(y1_test, w2) + B2
	test_prediction = tf.nn.softmax(test_logits)
	y2_test = tf.nn.relu(tf.matmul(tf_test_dataset2, w1) + B1)
	test_logits2 = tf.matmul(y2_test, w2) + B2
	test_prediction2 = tf.nn.softmax(test_logits2)
	print test_prediction2
	print test_prediction


# Let's run it:
num_steps = 3001

with tf.Session(graph=graph) as session:
	tf.initialize_all_variables().run()
	print("Initialized")
	for step in range(num_steps):
		# Pick an offset within the training data, which has been randomized.
		# Note: we could use better randomization across epochs.
		offset = (step * batch_size) % (tr_labels.shape[0] - batch_size)
		# Generate a minibatch.
		batch_data = tr_features[offset:(offset + batch_size), :]
		batch_labels = tr_labels[offset:(offset + batch_size), :]
		#print batch_labels
		# Prepare a dictionary telling the session where to feed the minibatch.
		# The key of the dictionary is the placeholder node of the graph to be fed,
		# and the value is the numpy array to feed to it.
		feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
		_, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
		if (step % 500 == 0):
			print np.shape(batch_data)
			print np.shape(predictions)           
			print np.shape(batch_labels)
			print("Minibatch loss at step %d: %f" % (step, l))
			print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
			#print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
			print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), ts_labels))
  			print("Test accuracy 2: %.1f%%" % accuracy(test_prediction2.eval(), single_labels))
	print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), ts_labels))
	print("Test accuracy 2: %.1f%%" % accuracy(test_prediction2.eval(), single_labels))    
	print test_prediction2.eval()
	print test_prediction.eval()

Tensor("Softmax_2:0", shape=(10700, 2), dtype=float64)
Tensor("Softmax_1:0", shape=(2700, 2), dtype=float64)
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 0: 4128.647461
Minibatch accuracy: 10.5%
Test accuracy: 80.6%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 500: 0.094453
Minibatch accuracy: 100.0%
Test accuracy: 70.3%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 1000: 0.242242
Minibatch accuracy: 93.4%
Test accuracy: 70.3%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 1500: 0.592992
Minibatch accuracy: 76.0%
Test accuracy: 70.3%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 2000: 0.083567
Minibatch accuracy: 100.0%
Test accuracy: 70.3%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 2500: 0.279398
Minibatch accuracy: 91.9%
Test accuracy: